# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

Make sure that you've downloaded the required human and dog datasets:
* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dogImages`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home directory, at location `/lfw`.  

*Note: If you are using a Windows machine, you are encouraged to use [7zip](http://www.7-zip.org/) to extract the folder.*

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [1]:
# Imports
import numpy as np
from glob import glob
import os
import cv2
import matplotlib.pyplot as plt                        
from tqdm import tqdm
from PIL import Image, ImageFile

import torch
import torchvision.models as models
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# the following import is required for training to be robust to truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print('Using CUDA', use_cuda)

def clear_gpu_memory(data):
    if use_cuda:
        for i in data:
            del i
        torch.cuda.empty_cache()

Using CUDA True


In [ ]:
# load filenames for human and dog images
human_files = np.array(glob("lfw/*/*"))
dog_files = np.array(glob("dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

<a id='step1'></a>
## Step 1: Detect Humans

In this section, we use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  

OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.  In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
%matplotlib inline                               

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[0])
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 
(You can print out your results and/or write your percentages in this cell)

In [ ]:
human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

## Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.
num_faces = 0
num_faces_in_dog_img = 0
for i in tqdm(range(100)):
    human = human_files_short[i]
    dog = dog_files_short[i]
    if face_detector(human):
        num_faces += 1            
    if face_detector(dog):
        num_faces_in_dog_img += 1

print('Human faces detected', num_faces)
print('Human faces detected in dog images', num_faces_in_dog_img)

We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

In [ ]:
### (Optional) 
### TODO: Test performance of another face detection algorithm.
### Feel free to use as many code cells as needed.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [ ]:
# define VGG16 model
VGG16 = models.vgg16(pretrained=True)
# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  The output should always be an integer between 0 and 999, inclusive.

Before writing the function, make sure that you take the time to learn  how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
# Set PIL to be tolerant of image files that are truncated.
ImageFile.LOAD_TRUNCATED_IMAGES = True

def process_img(img_path):
    '''
    Processes an image for input to a vgg16 net.
    Resizes so shortest side is 255 while preserving aspect ratio, crops the center 224x224,
    normalizes pixels, then returns it as a pytorch float tensor.
    
    Args:
        img_path: path to an image
    
    Returns:
        Pytorch Float tensor representing cropped and normalized image ready for pretrained pytorch model
    '''
    img = Image.open(img_path)
    width, height = img.size
    
    # resize so the shortest size is 255 pixels while preserving aspect ratio
    if width < height:
        img.resize((255, int(255 * height / width)))
    else:
        img.resize((int(255 * width / height), 255))
    
    # the new diemnsions
    width, height = img.size
    
    # center crop a 224x224 box
    left = (width - 224) / 2
    top = (height - 224) / 2 
    right = (width + 224) / 2
    bottom = (height + 224) / 2
    img = img.crop((left, top, right, bottom))
    
    img = np.array(img)
    # Make the color channel dimension first instead of last
    img = img.transpose((2, 0, 1)) # 3x224x224

    # convert all pixel values to between 0 and 1
    img = img / 255 
    # then normalize using means 0.485, 0.456, 0.406 and medians 0.229, 0.224, 0.225
    img[0] = (img[0] - 0.485) / 0.229 
    img[1] = (img[1] - 0.456) / 0.224
    img[2] = (img[2] - 0.406) / 0.225
    img = img[np.newaxis, : ] # create dimension for batch size of 1, so img.shape is 1x3x224x224
    
    # turn into tensor
    torch_img = torch.from_numpy(img)
    # turn into float tensor
    torch_img = torch_img.float()
    if use_cuda:
        torch_img = torch_img.cuda()
    return torch_img
    
    
def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''    
    processed_img = process_img(img_path)
    return VGG16(processed_img) # predicted class index

### (IMPLEMENTATION) Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

Use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    predictions =  VGG16_predict(img_path)
    _, index = predictions.topk(1, dim=1)
    isDog = 151 <= index <= 268
    return isDog

### (IMPLEMENTATION) Assess the Dog Detector

__Question 2:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 


In [ ]:
### Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
num_dogs = 0
num_mistaken_human_for_dog = 0
for i in tqdm(range(100)):
    human = human_files_short[i]
    dog = dog_files_short[i]  
    if dog_detector(dog):
        num_dogs += 1
    if dog_detector(human):
        num_mistaken_human_for_dog += 1
    

print('Dogs detected in dog images {}%'.format(num_dogs))
print('Humans that was deemed a dog {}%'.format(num_mistaken_human_for_dog))

We suggest VGG-16 as a potential network to detect dog images in your algorithm, but you are free to explore other pre-trained networks (such as [Inception-v3](http://pytorch.org/docs/master/torchvision/models.html#inception-v3), [ResNet-50](http://pytorch.org/docs/master/torchvision/models.html#id3), etc).  Please use the code cell below to test other pre-trained PyTorch models.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

In [ ]:
### (Optional) 
### TODO: Report the performance of another pre-trained network.
### Feel free to use as many code cells as needed.

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 10%.  In Step 4 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [3]:
### Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
train_dir = './dogImages/train'
valid_dir = './dogImages/valid'
test_dir = './dogImages/test'
batch_size = 20

train_transforms = transforms.Compose([transforms.RandomRotation(15),
                                        transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255), 
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255), 
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size)

loaders_scratch = {
    'train': train_loader, 
    'valid': valid_loader, 
    'test': test_loader
}

**Question 3:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**:

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  Use the template in the code cell below.

In [29]:
# 8 conv layers with batch normalization
class NetBN(nn.Module):
    def __init__(self):
        super(NetBN, self).__init__()
        
        conv1_out = 64
        conv2_out = 64
        conv3_out = 128
        conv4_out = 128
        conv5_out = 128
        conv6_out = 256
        conv7_out = 512
        conv8_out = 512
        
        num_classes = 133 # number of dog breeds
        self.fc_input_size = conv8_out * 7 * 7 # 7 is size of features after all max pooling layers
        
        self.conv1 = nn.Conv2d(3, conv1_out, 3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, 3, padding=1)
        self.conv3 = nn.Conv2d(conv2_out, conv3_out, 3, padding=1)
        self.conv4 = nn.Conv2d(conv3_out, conv4_out, 3, padding=1)
        self.conv5 = nn.Conv2d(conv4_out, conv5_out, 3, padding=1)
        self.conv6 = nn.Conv2d(conv5_out, conv6_out, 3, padding=1)
        self.conv7 = nn.Conv2d(conv6_out, conv7_out, 3, padding=1)
        self.conv8 = nn.Conv2d(conv7_out, conv8_out, 3, padding=1)

        self.conv1_bn = nn.BatchNorm2d(conv1_out)
        self.conv2_bn = nn.BatchNorm2d(conv2_out)
        self.conv3_bn = nn.BatchNorm2d(conv3_out)
        self.conv4_bn = nn.BatchNorm2d(conv4_out)
        self.conv5_bn = nn.BatchNorm2d(conv5_out)
        self.conv6_bn = nn.BatchNorm2d(conv6_out)
        self.conv7_bn = nn.BatchNorm2d(conv7_out)
        self.conv8_bn = nn.BatchNorm2d(conv8_out)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(self.fc_input_size, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3_out = nn.Linear(4096, num_classes)  
        
        self.dropout = nn.Dropout(0) # start with p=0 initially. BatchNorm reduces the need for dropout.
        
    def forward(self, x):
        # input x: 20x3x224x224 (batch_size x depth x width x height)
        # CONV -> BN -> ReLU -> MaxPool -> FC w ReLU -> Linear output
        
        # Convolutional layers
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = self.pool(x)
        
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = self.pool(x)
        
        x = F.relu(self.conv3_bn(self.conv3(x)))
        x = F.relu(self.conv4_bn(self.conv4(x))) 
        x = self.pool(x)           
        
        x = F.relu(self.conv5_bn(self.conv5(x)))
        x = F.relu(self.conv6_bn(self.conv6(x))) 
        x = self.pool(x)        
        
        x = F.relu(self.conv7_bn(self.conv7(x)))
        x = F.relu(self.conv8_bn(self.conv8(x))) 
        x = self.pool(x)           
        
        # flatten for FC layers
        x = x.view(-1, self.fc_input_size)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        # using CrossEntropyLoss which combines LogSoftmax and NLLLoss, so no activation function here
        x = self.fc3_out(x) 
        return x

In [5]:
# 8 conv layers with batch normalization
class NetBN_Deep(nn.Module):
    def __init__(self, num_classes):
        super(NetBN_Deep, self).__init__()
        conv1_out = 64
        conv2_out = 64
        conv3_out = 64
        conv4_out = 64
        
        conv5_out = 128
        conv6_out = 128
        conv7_out = 128
        
        conv8_out = 128
        conv9_out = 256
        conv10_out = 256
        
        conv11_out = 256
        conv12_out = 256
        conv13_out = 512
        
        conv14_out = 512
        conv15_out = 512
        conv16_out = 512
        
        self.conv1 = nn.Conv2d(3, conv1_out, 3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, 3, padding=1)
        self.conv3 = nn.Conv2d(conv2_out, conv3_out, 3, padding=1)
        self.conv4 = nn.Conv2d(conv3_out, conv4_out, 3, padding=1)
        
        self.conv5 = nn.Conv2d(conv4_out, conv5_out, 3, padding=1)
        self.conv6 = nn.Conv2d(conv5_out, conv6_out, 3, padding=1)
        self.conv7 = nn.Conv2d(conv6_out, conv7_out, 3, padding=1)
        
        self.conv8 = nn.Conv2d(conv7_out, conv8_out, 3, padding=1)
        self.conv9 = nn.Conv2d(conv8_out, conv9_out, 3, padding=1)
        self.conv10 = nn.Conv2d(conv9_out, conv10_out, 3, padding=1)
        
        self.conv11 = nn.Conv2d(conv10_out, conv11_out, 3, padding=1)
        self.conv12 = nn.Conv2d(conv11_out, conv12_out, 3, padding=1)
        self.conv13 = nn.Conv2d(conv12_out, conv13_out, 3, padding=1)
        
        self.conv14 = nn.Conv2d(conv13_out, conv14_out, 3, padding=1)
        self.conv15 = nn.Conv2d(conv14_out, conv15_out, 3, padding=1)
        self.conv16 = nn.Conv2d(conv15_out, conv16_out, 3, padding=1)

        self.bn64 = nn.BatchNorm2d(64)
        self.bn128 = nn.BatchNorm2d(128)
        self.bn256 = nn.BatchNorm2d(256)
        self.bn512 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc_input_size = conv16_out * 7 * 7 # 7 is size of features after all max pooling
        self.fc1 = nn.Linear(self.fc_input_size, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3_out = nn.Linear(4096, num_classes)  
        
        self.dropout = nn.Dropout(0) # start with p=0 initially. BatchNorm reduces the need for dropout.
        
    def forward(self, x):
        # input x: 20x3x224x224 (batch_size x depth x width x height)
        
        # Convolutional layers
        x = F.relu(self.bn64(self.conv1(x)))
        x = F.relu(self.bn64(self.conv2(x)))
        x = F.relu(self.bn64(self.conv3(x)))
        x = F.relu(self.bn64(self.conv4(x)))
        x = self.pool(x)
        
        x = F.relu(self.bn128(self.conv5(x)))
        x = F.relu(self.bn128(self.conv6(x)))
        x = F.relu(self.bn128(self.conv7(x)))
        x = self.pool(x)

        x = F.relu(self.bn128(self.conv8(x)))
        x = F.relu(self.bn256(self.conv9(x)))
        x = F.relu(self.bn256(self.conv10(x)))
        x = self.pool(x)
        
        x = F.relu(self.bn256(self.conv11(x)))
        x = F.relu(self.bn256(self.conv12(x)))
        x = F.relu(self.bn512(self.conv13(x)))
        x = self.pool(x)
        
        x = F.relu(self.bn512(self.conv14(x)))
        x = F.relu(self.bn512(self.conv15(x)))
        x = F.relu(self.bn512(self.conv16(x)))
        x = self.pool(x)
        
        # flatten for FC layers
        x = x.view(-1, self.fc_input_size)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        # using CrossEntropyLoss which combines LogSoftmax and NLLLoss, so no activation function here
        x = self.fc3_out(x) 
        
        return x

In [ ]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], 
                      x.shape[1] // self._pool_size,
                      self._pool_size,
                      *x.shape[2:]).max(2)
        return m

In [ ]:
# 8 conv layers with batch normalization and maxout instead of relu
class NetBN_MO(nn.Module):
    def __init__(self):
        super(NetBN_MO, self).__init__()
        
        conv1_out = 64
        conv2_out = 64
        conv3_out = 128
        conv4_out = 128
        conv5_out = 128
        conv6_out = 256
        conv7_out = 512
        conv8_out = 512
        
        num_classes = 133 # number of dog breeds
        self.fc_input_size = conv8_out * 7 * 7 # 7 is size of features after all max pooling layers
        
        self.conv1 = nn.Conv2d(3, conv1_out, 3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, 3, padding=1)
        self.conv3 = nn.Conv2d(conv2_out, conv3_out, 3, padding=1)
        self.conv4 = nn.Conv2d(conv3_out, conv4_out, 3, padding=1)
        self.conv5 = nn.Conv2d(conv4_out, conv5_out, 3, padding=1)
        self.conv6 = nn.Conv2d(conv5_out, conv6_out, 3, padding=1)
        self.conv7 = nn.Conv2d(conv6_out, conv7_out, 3, padding=1)
        self.conv8 = nn.Conv2d(conv7_out, conv8_out, 3, padding=1)

        self.conv1_bn = nn.BatchNorm2d(conv1_out)
        self.conv2_bn = nn.BatchNorm2d(conv2_out)
        self.conv3_bn = nn.BatchNorm2d(conv3_out)
        self.conv4_bn = nn.BatchNorm2d(conv4_out)
        self.conv5_bn = nn.BatchNorm2d(conv5_out)
        self.conv6_bn = nn.BatchNorm2d(conv6_out)
        self.conv7_bn = nn.BatchNorm2d(conv7_out)
        self.conv8_bn = nn.BatchNorm2d(conv8_out)
        
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(self.fc_input_size, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3_out = nn.Linear(4096, num_classes)  
        
        self.dropout = nn.Dropout(0) # start with p=0 initially. BatchNorm reduces the need for dropout.
        
    def forward(self, x):
        # input x: 20x3x224x224 (batch_size x depth x width x height)
        # CONV -> BN -> Maxout -> MaxPool -> FC w ReLU -> Linear output
        
        # Convolutional layers
        x = self.conv1(x)
        x = self.conv1_bn(x)
        x = self.pool(x)
        x = self.maxout(x)
        
        x = self.conv2(x)
        x = self.conv2_bn(x)
        x = self.pool(x)
        x = self.maxout(x)
        
        x = self.conv3(x)
        x = self.conv3_bn(x)
        x = self.maxout(x)
        x = self.conv4(x)
        x = self.conv4_bn(x)
        x = self.maxout(x)
        x = self.pool(x)           
        
        x = self.conv5(x)
        x = self.conv5_bn(x)
        x = self.maxout(x)
        x = self.conv6(x)
        x = self.conv6_bn(x)
        x = self.maxout(x)
        x = self.pool(x)           
        
        x = self.conv7(x)
        x = self.conv7_bn(x)
        x = self.maxout(x)
        x = self.conv8(x)
        x = self.conv8_bn(x)
        x = self.maxout(x)
        x = self.pool(x)                      
        
        # flatten for FC layers
        x = x.view(-1, self.fc_input_size)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        # using CrossEntropyLoss which combines LogSoftmax and NLLLoss, so no activation function here
        x = self.fc3_out(x) 
        return x
    def maxout(self, x):
        pass

In [30]:
# instantiate the CNN
model_scratch = NetBN()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

model_scratch

NetBN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_s

__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__ 


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [33]:
learning_rate = 0.15
criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.SGD(params=model_scratch.parameters(), lr=learning_rate)

Here I defined model save and load functions, saving the optimizer state along with the model's parameters to allow for training continuation.

In [89]:
def save_model(model_name, model_state_dict, optim_state_dict, 
               accuracy, epoch, lr, min_valid_loss, save_path):
    checkpoint = {
        "model_name": model_name,
        "model_state_dict": model_state_dict,
        "optim_state_dict": optim_state_dict,
        "validation_accuracy": accuracy,
        "epoch": epoch,
        "learningrate": lr,
        "min_valid_loss": min_valid_loss
    }
    torch.save(checkpoint, save_path)
    print("Model saved")
    
def load_model(model, optim, checkpoint_name):
    try:
        checkpoint = torch.load(checkpoint_name)
        model.load_state_dict(checkpoint['model_state_dict'])
        optim.load_state_dict(checkpoint['optim_state_dict'])
        accuracy = checkpoint["validation_accuracy"]
        min_valid_loss = checkpoint["min_valid_loss"]
        epoch = checkpoint['epoch'] # left off on epoch #
        print("Loaded a model. \tAccuracy: {} \tMin valid loss: {}".format(accuracy.item(), min_valid_loss))
        print('Achieved on epoch ', epoch)
        return epoch, min_valid_loss     
    except FileNotFoundError:
        print('Model checkpoint does not exist')

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `save_path`.

In [27]:
save_path_bn = 'model_scratch_BN.pt'
save_path_bn_deep = 'model_scratch_BN_deep.pt'
save_path_bn_mo = 'model_scratch_BN_Maxout.pt'
save_path = save_path_bn

In [12]:
def change_learning_rate(optimizer, by_factor):
    for param_group in optimizer.param_groups:
        previous_lr = param_group['lr']
        param_group['lr'] *= by_factor
        print("Changed LR: {} -> {}".format(previous_lr, param_group['lr']))
        
def change_dropout_intensity(model, intensity=0):
    if 0 < intensity < 1:
        model.dropout.p = intensity
        print('Set dropout to ', intensity)
    elif model.dropout.p < 0.5:
        if model.dropout.p == 0:
            model.dropout.p += 0.1
        else:
            model.dropout.p += 0.05
        print('Increasing dropout to ', model.dropout.p)
    else:
        print('Maximum dropout reached')

In [98]:
def train(n_epochs, loaders, model, optimizer, criterion, 
          use_cuda, save_path, scheduler, resume_epoch=1, valid_loss_min = np.Inf):
    """returns trained model"""
    print('Training model', save_path)
    auto_change_dropout = False

    for epoch in range(resume_epoch, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        accuracy = 0.0
        correct = 0.
        total = 0.

        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)
            clear_gpu_memory([data, target, output, loss])

            if (batch_idx + 1) % batch_size == 0:
                print('Batch: {}, Training Loss: {:.6f}'
                      .format(batch_idx + 1, train_loss))

        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            with torch.no_grad():
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                ## update the average validation loss
                output = model(data)
                loss = criterion(output, target)
                valid_loss += (1 / (batch_idx + 1)) * (loss.data - valid_loss)

                # convert output probabilities to predicted class
                pred = output.data.max(1, keepdim=True)[1]
                # compare predictions to true label
                correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
                total += data.size(0)

                clear_gpu_memory([data, target])
        
        if scheduler:
            scheduler.step(valid_loss)
            for param_group in optimizer.param_groups:
                print('LR:', param_group['lr'])
            
        
        # increase dropout
        if auto_change_dropout and train_loss - valid_loss < 0.15: 
            print('Auto increasing dropout')
            change_dropout_intensity(model)
            if model.dropout.p > 0.2:
                auto_change_dropout = False

        # print training/validation statistics
        accuracy = 100. * correct / total
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}\nValid Accuracy: {:.3f}% ({}/{})'.format(
            epoch, 
            train_loss,
            valid_loss,
            accuracy,
            correct, total
        ))

        ## save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Saving model')
            save_model(model_name='FromScratch', 
                       model_state_dict=model.state_dict(), 
                       optim_state_dict=optimizer.state_dict(), 
                       accuracy=accuracy, lr=learning_rate, epoch=epoch, min_valid_loss=valid_loss,
                       save_path=save_path)
            valid_loss_min = valid_loss
        print('\n')
    # return trained model
    return model

In [ ]:
continue_training = False
try:
    if continue_training:
        print('Training continued')
        resume_epoch, min_valid_loss = load_model(model_scratch, optimizer_scratch, save_path)    

        #change_dropout_intensity(model_scratch, 0.4)
        change_learning_rate(optimizer_scratch, 0.4)

        model_scratch = train(100, 
                              loaders_scratch, model_scratch, 
                              optimizer_scratch, criterion_scratch, 
                              use_cuda, save_path, 
                              resume_epoch = resume_epoch, valid_loss_min=min_valid_loss)
    else:
        # train the model
        print('Training started')
        model_scratch = train(100, loaders_scratch, model_scratch, 
                              optimizer_scratch, criterion_scratch, 
                              use_cuda, save_path)

    # load the model that got the best validation accuracy
    load_model(model_scratch, optimizer_scratch, save_path)
except KeyboardInterrupt:
    print('Training interrupted')

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images.  Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 10%.

In [34]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)

        if use_cuda:
            del data
            del target
            torch.cuda.empty_cache()
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

    
# load the model that got the best validation accuracy
load_model(model_scratch, optimizer_scratch, save_path)
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Loaded a model. 	Accuracy: 26.46706586826347 	Min valid loss: 2.8255250453948975
Achieved on epoch  42
Dropout intensity 0
Test Loss: 2.778571


Test Accuracy: 27% (232/836)


---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively). 

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [35]:
loaders_transfer = loaders_scratch

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [106]:
import torchvision.models as models
import torch.nn as nn

feature_extracting = True

model_transfer = models.resnet50(pretrained=True)
model_transfer.fc.out_features = 133
if use_cuda:
    model_transfer = model_transfer.cuda()

for name, param in model_transfer.named_parameters():
    param.requires_grad = False    
    
model_transfer.fc.weight.requires_grad = True
model_transfer.fc.bias.requires_grad = True

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [107]:
learning_rate_transfer = 0.15
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(params=model_transfer.parameters(), lr=learning_rate_transfer)
scheduler_valid = optim.lr_scheduler.ReduceLROnPlateau(optimizer_transfer, factor=0.25, patience=1, verbose=True)
scheduler_train = optim.lr_scheduler.ReduceLROnPlateau(optimizer_transfer, mode='max', factor=0.25, patience=1, verbose=True)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [101]:
save_path = 'model_transfer_resnet50.pt'

In [108]:
try:
    model_transfer = train(100, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, 
                           use_cuda, save_path, schedulers={'valid': scheduler_valid, 'train': scheduler_train})
except KeyboardInterrupt:
    print('Training interrupted.')
# load the model that got the best validation accuracy (uncomment the line below)
load_model(model_transfer, optimizer_transfer, save_path)

Training model model_transfer_resnet50.pt
Batch: 20, Training Loss: 8.858770
Batch: 40, Training Loss: 8.203654
Batch: 60, Training Loss: 7.687531
Batch: 80, Training Loss: 7.187061
Batch: 100, Training Loss: 6.565994
Batch: 120, Training Loss: 6.104435
Batch: 140, Training Loss: 5.798542
Batch: 160, Training Loss: 5.478411
Batch: 180, Training Loss: 5.169446
Batch: 200, Training Loss: 4.898210
Batch: 220, Training Loss: 4.689732
Batch: 240, Training Loss: 4.495698
Batch: 260, Training Loss: 4.326474
Batch: 280, Training Loss: 4.159091
Batch: 300, Training Loss: 4.022431
Batch: 320, Training Loss: 3.916486
LR: 0.15
Epoch: 1 	Training Loss: 3.849501 	Validation Loss: 0.946184
Valid Accuracy: 69.940% (584.0/835.0)
Saving model
Model saved


Batch: 20, Training Loss: 1.721227
Batch: 40, Training Loss: 1.773940
Batch: 60, Training Loss: 1.650665
Batch: 80, Training Loss: 1.622813
Batch: 100, Training Loss: 1.626204
Batch: 120, Training Loss: 1.651206
Batch: 140, Training Loss: 1.684904
Bat

Batch: 260, Training Loss: 0.705491
Batch: 280, Training Loss: 0.699406
Batch: 300, Training Loss: 0.701197
Batch: 320, Training Loss: 0.698495
LR: 0.0375
Epoch: 12 	Training Loss: 0.697363 	Validation Loss: 0.415610
Valid Accuracy: 86.946% (726.0/835.0)
Saving model
Model saved


Batch: 20, Training Loss: 0.603090
Batch: 40, Training Loss: 0.639195
Batch: 60, Training Loss: 0.678312
Batch: 80, Training Loss: 0.689208
Batch: 100, Training Loss: 0.700223
Batch: 120, Training Loss: 0.722002
Batch: 140, Training Loss: 0.723837
Batch: 160, Training Loss: 0.728102
Batch: 180, Training Loss: 0.729715
Batch: 200, Training Loss: 0.724031
Batch: 220, Training Loss: 0.730043
Batch: 240, Training Loss: 0.736577
Batch: 260, Training Loss: 0.733357
Batch: 280, Training Loss: 0.737349
Batch: 300, Training Loss: 0.735083
Batch: 320, Training Loss: 0.726935
LR: 0.0375
Epoch: 13 	Training Loss: 0.728092 	Validation Loss: 0.441743
Valid Accuracy: 87.425% (730.0/835.0)


Batch: 20, Training Loss: 0.64541

Batch: 80, Training Loss: 0.633945
Batch: 100, Training Loss: 0.627375
Batch: 120, Training Loss: 0.627473
Batch: 140, Training Loss: 0.630793
Batch: 160, Training Loss: 0.628284
Training interrupted.
Loaded a model. 	Accuracy: 88.50299401197604 	Min valid loss: 0.3696597218513489
Achieved on epoch  21


(21, tensor(0.3697, device='cuda:0'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [109]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

Test Loss: 0.403361


Test Accuracy: 88% (741/836)


### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    return None

---
<a id='step5'></a>
## Step 5: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `dog_detector` functions developed above.  You are __required__ to use your CNN from Step 4 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](images/sample_human_output.png)


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    


---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ (Three possible points for improvement)

In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.

## suggested code, below
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)